###### 2020-11-23 월요일

# 02_Bidirect LSTM + K-fold
 - 구글 collab으로 실행하였습니다

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [87]:
import pandas as pd
import numpy as np
import re
import json

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold, train_test_split

## 1. 데이터불러오기 & 합치기

In [5]:
feature = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/02_K-Fold_O/data/feature.csv', index_col=[0])
target = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/02_K-Fold_O/data/target.csv', index_col=[0])

word_index_json = open('/content/drive/MyDrive/[final_project]_악플원정대/02_K-Fold_O/data/word_index_vocab.json', 'r').read()
word_index_vocab = json.loads(word_index_json)

## 2. 데이터토큰화 & 패딩

#### 텍스트 음절단위로 자르기

In [6]:
feature_split = feature['댓글'].apply(list).tolist()

#### 정수인덱싱

In [7]:
tokenizer = Tokenizer()
tokenizer.word_index = word_index_vocab

feature_sequences = tokenizer.texts_to_sequences(feature_split)

### 패딩

In [8]:
feature_input = pad_sequences(feature_sequences, padding='post', maxlen=400)

## 3. SMOTE를 이용한 라벨 불균형 해소

In [9]:
target.value_counts()

악플여부
0       67399
1       14225
dtype: int64

 - 라벨 0은 446162, 1은 2897, 2는 4993으로 균형이 잡혀있지않아 recall(재현율)이 떨어질 가능성이 높다
 - 그래서 SMOTE를 이용하여 라벨이 1과 2인 경우를 복제하여 라벨학습의 균형을 맞추어 주는 것이다
 - 이것을 `오버샘플링`이라 한다.

In [10]:
from imblearn.over_sampling import SMOTE

In [11]:
smote = SMOTE(random_state=123)
feature_over, target_over = smote.fit_sample(feature_input, target)

In [12]:
pd.Series(target_over).value_counts()

1    67399
0    67399
dtype: int64

In [13]:
target_over = target_over.reshape(-1, 1)

## 4. Bidirect LSTM + K-Fold 구현

In [60]:
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import f1_score

In [83]:
vocab_size = len(word_index_vocab)
embedding_dim = 16
max_length = 400
n_class = 2

In [84]:
# LSTM 모델 생성

def get_model():

    from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

    model = Sequential()
    model.add(Embedding(vocab_size + 1, embedding_dim, input_length=max_length))
    model.add(Bidirectional(LSTM(16, return_sequences=True)))
    model.add(Bidirectional(LSTM(16, return_sequences=False)))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [85]:
# 3-Fold 시행
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=3, random_state=123, shuffle=True)

In [89]:
validation_pred = np.zeros((feature_over.shape[0], 1))


k = 0
for train_idx, val_idx in tqdm_notebook(cv.split(feature_over, target_over)) :

    print('{}번째 Fold 시작' .format(k+1))

    x_train = feature_over[train_idx]
    y_train = target_over[train_idx]

    x_validation = feature_over[val_idx]
    y_validation = target_over[val_idx]

    LSTM = get_model()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

    LSTM.fit(x_train, y_train,
             callbacks          = [es],
             epochs             = 20,
             batch_size         = 64,
             validation_data    = (x_validation, y_validation))
    
    validation_pred[val_idx, : ] = LSTM.predict(x_validation) 
    print('')

    k += 1

1번째 Fold 시작
Epoch 1/20
1405/1405 [==============================] - 104s 74ms/step - loss: 0.4041 - acc: 0.8187 - val_loss: 0.3785 - val_acc: 0.8378
Epoch 2/20
1405/1405 [==============================] - 100s 71ms/step - loss: 0.3378 - acc: 0.8576 - val_loss: 0.3504 - val_acc: 0.8500
Epoch 3/20
1405/1405 [==============================] - 98s 70ms/step - loss: 0.3077 - acc: 0.8728 - val_loss: 0.3308 - val_acc: 0.8621
Epoch 4/20
1405/1405 [==============================] - 97s 69ms/step - loss: 0.2846 - acc: 0.8849 - val_loss: 0.2883 - val_acc: 0.8818
Epoch 5/20
1405/1405 [==============================] - 97s 69ms/step - loss: 0.2640 - acc: 0.8946 - val_loss: 0.2664 - val_acc: 0.8932
Epoch 6/20
1405/1405 [==============================] - 96s 69ms/step - loss: 0.2329 - acc: 0.9089 - val_loss: 0.2430 - val_acc: 0.9043
Epoch 7/20
1405/1405 [==============================] - 96s 69ms/step - loss: 0.2713 - acc: 0.8929 - val_loss: 0.2714 - val_acc: 0.8965
Epoch 8/20
1405/1405 [============

In [48]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score,  confusion_matrix

## 5. 모델 평가

In [52]:
def model_evaluate(y_pred, y_test):
    print('accuracy_score : ', accuracy_score(y_pred, y_test))
    print('recall         : ', recall_score(y_pred, y_test))
    print('precision      :', precision_score(y_pred, y_test))
    print('Confusion Matrix : \n', confusion_matrix(y_pred, y_test))

In [53]:
pred = validation_pred > 0.5

In [54]:
model_evaluate(pred, target_over)

ValueError: ignored

## 6. 최종모델 학습 및 테스트

In [57]:
final_model = get_model()

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
ms = ModelCheckpoint('LSTM_best.h5', monitor='val_loss', mode='min', save_best_only=True)


final_model.fit(feature_over, target_over, 
                callbacks                 = [es, ms],
                epochs                    = 20,
                batch_size                = 64,
                validation_split          = 0.2)


Epoch 1/20
1685/1685 [==============================] - 114s 68ms/step - loss: 0.3819 - acc: 0.8359 - val_loss: 0.3551 - val_acc: 0.8420
Epoch 2/20
1685/1685 [==============================] - 112s 66ms/step - loss: 0.2910 - acc: 0.8834 - val_loss: 0.2567 - val_acc: 0.8811
Epoch 3/20
1685/1685 [==============================] - 111s 66ms/step - loss: 0.2492 - acc: 0.9035 - val_loss: 0.2683 - val_acc: 0.8815
Epoch 4/20
1685/1685 [==============================] - 111s 66ms/step - loss: 0.2223 - acc: 0.9151 - val_loss: 0.2354 - val_acc: 0.8988
Epoch 5/20
1685/1685 [==============================] - 111s 66ms/step - loss: 0.1974 - acc: 0.9259 - val_loss: 0.1524 - val_acc: 0.9364
Epoch 6/20
1685/1685 [==============================] - 111s 66ms/step - loss: 0.1850 - acc: 0.9313 - val_loss: 0.1903 - val_acc: 0.9242
Epoch 7/20
1685/1685 [==============================] - 110s 65ms/step - loss: 0.1679 - acc: 0.9377 - val_loss: 0.1978 - val_acc: 0.9175
Epoch 8/20
1685/1685 [===================

In [76]:
loaded_model = load_model('LSTM_best.h5')

In [77]:
def model_test(comment, threshold) :
    comment_list = [list(comment)]
    comment_label = tokenizer.texts_to_sequences(comment_list)
    comment_pad = pad_sequences(comment_label, padding='post', maxlen=400)
    pred = loaded_model.predict(comment_pad)

    if pred < threshold :
        result = '악플이 아닙니다'
    else :
        result = '악플 입니다'

    print(result)
    print('라벨확률 :', pred)

    return pred

In [82]:
comment = ''''''
threshold = 0.5

prob = model_test(comment, threshold)

악플 입니다
라벨확률 : [[0.979735]]
